<br><strong>Coding for Venezuela - Team Penguin</strong>
<br><strong>Track 2 - Dashboarding for NGOs</strong>

Team Penguin: Duy Pham, Linh Le, Paolo Schirru, Peter Huesmann, Skander Driss <br>

<h3>Instructions</h3><br>

This program is using bigquery raw data tables in Angostura to create a dataframe that transformed through data cleaning and feature engineering in Python and visualised in Tableau. The Python script can be run on Airflow and is available on github and devpost.
After running the python script a csv file is created that has to be imported into Tableu. 
The Python script is soft-coded and does not require changes if additional data is added.

<h4>Instructions on cleaning the data</h4><br>
    
The data is cleaned automatically when the script is run and saved as a new local csv file.
The data is cleaned in multiple steps:
    1. Creating Missing Value columns
    2. Changing the report date formating
    3. Imputing values used for inconsistency analysis
    4. Creating logical inconsistency analysis for data quality
    5. Creating a strike free column
    6. Creating a duplicate indicator column for data quality
    7. Removing duplicates from dataset
    8. Imputing missing values
    9. Creating missing values percentage for data quality
    10. Creating data quality index
    11. Ordinal encoding columns for Tableau
    12. Exporting to csv

<h4><strong>Instructions on generating dashboards based on the scripts</strong><br>
    
The Tableau files for the dashboard are available on github and devpost.
The live version of the dashboard can be reached here:
<https://public.tableau.com/profile/duy8257#!/vizhome/Venezuela_15818927364190/NGOdashboard?publish=yes>

 

<h3>Data Exploration</h3><br>

<strong>Domain Research</strong><br>

Since 2013, Venezuela has been experiencing a humanitarian crisis as a result of socioeconomic and political tensions. This has caused widespread poverty and massive shortages of basic necessities, including food and healthcare. National blackouts and power failures, as well as vaccine, equipment, and medication shortages have caused mortality rates to soar. Over 4 million people, or about 10% of Venezuela’s total population, have fled the country. About 30% of medical professionals leave the country, particularly young doctors. 
Government data is virtually non-existent, which is where the Angostura Data Platform comes in. This is a data collection and storage platform, allowing hospital personnel across the region to input data weekly, and NGOs focusing on the Venezuelan crisis to make effective, data-driven decisions.<br><br>

<strong>Data Exploration</strong><br><br>
Looking for variables related to supply, equipment, and operability as key factors for NGOs. The NGOs require  data to decide on how to allocate their budget and resources. They need information on the requirements and potential impact for each hospital and region. They need to be able to filter technical topics in the dashboards to allocate specific budgets for different solutions, such as the needs for specific medicine and equipment. They also need to be able to filter for time, hospital and region. After exploring the data we decided to focus on the scarcity of supplies and equipment, operability of services and data quality. Descriptions are given in the comments on feature engineering.

<h3>Setup</h3><br>

Setup and data loading for the data cleaning and feature engineering

In [8]:
################################################################################
# Import Packages
################################################################################

import pandas as pd                                   # data science essentials
import matplotlib.pyplot as plt                       # essential graphical output                                 # enhanced graphical output
import statsmodels.formula.api as smf                 # regression modeling
import numpy as np                                    # Math
from scipy import stats
import seaborn
import time

# setting options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
%matplotlib inline

<strong>Authentication Setup</strong>

Before being able to query the Angostura database you need to authenticate.
You will have read-only access to the tables.

In [9]:
# First we need to get the JSON key for authentication

import urllib.request

url = 'https://storage.googleapis.com/angostura-public/hult-hackathon-key.json'
urllib.request.urlretrieve(url, './hult-hackathon-key.json')

# Second, we add the key to our environment

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './hult-hackathon-key.json'

<strong>Import GCloud API</strong>

In [10]:
from google.cloud import bigquery

client = bigquery.Client()

<strong>Query</strong>

In [11]:
# We can dump the BigQuery results directly into a DataFrame
QUERY = ('select * from `angostura_dev`.eh_health_survey_response')

df = client.query(QUERY).to_dataframe()

In [13]:
df.to_csv("presentation.csv")

In [15]:
df = pd.read_csv("presentation.csv")

<h3>Data Cleaning and Feature Engineering</h3><br>


<h4>Creating missing value flags</h4><br>
We start by flagging all the missing values and creating missing value columns as a binary 1/0 if a column has a missing value.

In [16]:
################################################################################
# Feature Engineering 
################################################################################

# looping over columns with missing values
for col in df:

    # creating columns with 1s if missing and 0 if not
    if df[col].isnull().astype(int).sum() > 0:
        df['m_'+col] = df[col].isnull().astype(int)


<h4>Changing report date</h4><br>
Changing report week into a more readable format and setting up duplicate removal


In [17]:
################################################################################
# Support Functions
################################################################################

# Changing Report week to date time column
def week_to_datetime (data, col):
    report_date = data[col].str.replace(' del ', '_')
    data['report_date'] = pd.to_datetime('0_'+ report_date ,format='%w_%W_%Y', dayfirst='True')
    return data

# Dropping duplicates
def later_duplicated_hospital (data):
    data.drop_duplicates(['report_date', 'hospital_code'],keep='last',inplace = True)
    return data

In [18]:
# Using Week to date time function
week_to_datetime (df, 'report_week')

,Unnamed: 0,timestamp,report_week,operability_icu,power_outage_equipment_failure_specify,power_generator_available,power_outage_mortatility,power_outage_deaths_count,mortality_hospital_failure_cardiovascular_count,mortality_hospital_failure_cardiovascular_cause,mortality_hospital_failure_trauma_count,mortality_hospital_failure_trauma_cause,strike_medical_staff_affected,strike_nurses_affected,operability_icu_p,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,epidemiological_emergency_suspected_diseases,rrt_avail_high_flow_catheters,rrt_avail_blood_tests_hiv_hvb_hvc_vdr,rrt_avail_immediate_access_urea_reduction_bun,nCoV_face_mask_avail,nCoV_respiratory_isolation_protocol_avail,operability_er,nCoV_isolation_area_avail,operability_sx,operability_lab,operability_uls,operability_ct_mri,operability_xr,er_avail_adrenalin,er_avail_atropine,hospital_code,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,federal_entity,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,sx_avail_minor_opioids,sx_avail_major_opioids,hospital_type,sx_avail_anesthetic_gases,sx_avail_anesthetics_iv,sx_avail_relaxants,sx_avail_ott_intubation,sx_avail_patient_lingerie_kit,sx_avail_disposables_mask_gloves_gown,sx_avail_oxygen_suction,rrt_avail,rrt_operability,rrt_date_stopped_operability,administrative_entity,rrt_reason_stopped_operability,rrt_avg_daily_patients,rrt_peritoneal_count,rrt_peritoneal_reason_not_performed,rrt_num_hemodialysis,rrt_num_hemodialysis_acute,rrt_num_hemodialysis_chronic,rrt_num_hemodialysis_equipments,rrt_num_hemodialysis_equipments_operability,rrt_hemodialysis_avail_filter,arch_beds_count,rrt_hemodialysis_avail_lines,rrt_hemodialysis_avail_kit_hemodialysis,rrt_hemodialysis_avail_iron,rrt_hemodialysis_avail_b_complex,rrt_hemodialysis_avail_calcium,rrt_hemodialysis_avail_zemblar,rrt_reverse_osmosis_unit_operability,rrt_staff_nephrology,rrt_staff_md,rrt_staff_resident,op_beds_count,rrt_staff_nurse,rrt_staff_nurse_nephrologist,nutr_avail,nutr_operability,nutr_date_stopped_operability,nutr_reason_stopped_operability,nutr_num,nutr_daily_freq_meal,nutr_quality,nutr_freq_milk_formulas,op_beds_er_count,pneumonia_antibiotic_therapy_latency,mi_thrombolytic_treatment_latency,er_staff_residents_and_rural_day_on_call,er_staff_specialist_day_on_call,er_staff_mic_day_on_call,er_staff_nurse_day_on_call,er_staff_non_professional_nurse_day_on_call,er_staff_residents_and_rural_night_on_call,er_staff_specialist_night_on_call,er_staff_mic_night_on_call,op_pavilions_count,er_staff_nurse_night_on_call,er_staff_non_professional_nurse_night_on_call,wash_failure_icu,wash_failure_er,wahs_failure_sx,power_outage,power_outage_avg_failures_per_day,power_outage_days_count,power_outage_avg_duration,power_outage_equipment_failure,m_power_outage_equipment_failure_specify,m_power_outage_deaths_count,m_mortality_hospital_failure_cardiovascular_count,m_mortality_hospital_failure_cardiovascular_cause,m_mortality_hospital_failure_trauma_count,m_mortality_hospital_failure_trauma_cause,m_strike_medical_staff_affected,m_strike_nurses_affected,m_strike_other_staff_affected,m_strike_patients_affected,m_strike_other_affected,m_staff_violence_affected_reasons,m_epidemiological_emergency_suspected_diseases,m_rrt_avail_high_flow_catheters,m_rrt_avail_blood_tests_hiv_hvb_hvc_vdr,m_rrt_avail_immediate_access_urea_reduction_bun,m_nCoV_face_mask_avail,m_nCoV_respiratory_isolation_protocol_avail,m_nCoV_isolation_area_avail,m_rrt_operability,m_rrt_date_stopped_operability,m_rrt_reason_stopped_operability,m_rrt_avg_daily_patients,m_rrt_peritoneal_count,m_rrt_peritoneal_reason_not_performed,m_rrt_num_hemodialysis,m_rrt_num_hemodialysis

<h4>Imputing values for columns required in the inconsistency analysis</h4><br>

The Inconsistency analysis looks for inconsistencies in the input of different data columns to look for miss inputs that indicate a lower data quality.
It looks at the mortality causes columns for cardiovascular and trauma, as well as the mortality column for power outage mortality and compares them with their respective count columns. The cause has to be typed in by hand. It is more likely that a doctor typed in the cause of death and forgot to give the count, than it is, that he gave a cause of death when no deaths had occurred. Since a lot of the data has a cause, but no count for deaths this is a logical inconsistency. These inconsistencies are marked and calculated as part of the Quality index.

In [19]:
# Counting number of observation before removing duplicates and saving it for use in
# inconsistency analysis
lenght1 = len(df)

In [20]:
# Imputing 'mortality_hospital_failure_cardiovascular_count'
# We can assume that most hospitals without cases simply left this field blank and can therfore imput
# '0'
# This column and mortality_hospital_failure_cardiovascular_cause are correlated. However in some cases
# a cause of mortality is listed but no count is given. This most likely is due to incorrect data
# gathering. We could assume a number of 1 in this case or leave it at zero. 
fill = 0
df['mortality_hospital_failure_cardiovascular_count'] = df['mortality_hospital_failure_cardiovascular_count'].fillna(fill)

In [21]:
# Imputing 'mortality_hospital_failure_cardiovascular_cause'
# If there was a count we can impute unkown, if there is no count we can impute no deaths or 0
fill = 'unknown'
df['mortality_hospital_failure_cardiovascular_cause'] = df['mortality_hospital_failure_cardiovascular_cause'].fillna(fill)

In [22]:
# Imputing 'mortality_hospital_failure_trauma_count'
# We can assume that most hospitals without cases simply left this field blank and can therfore imput
# '0'
# This column and mortality_hospital_failure_trauma_cause are correlated. However in some cases
# a cause of mortality is listed but no count is given. This most likely is due to incorrect data
# gathering. We could assume a number of 1 in this case or leave it at zero. 
fill = 0
df['mortality_hospital_failure_trauma_count'] = df['mortality_hospital_failure_trauma_count'].fillna(fill)

In [23]:
# Imputing 'mortality_hospital_failure_trauma_cause'
# If there was a count we can impute unkown, if there is no count we can impute no deaths or 0
fill = 'unknown'
df['mortality_hospital_failure_trauma_cause'] = df['mortality_hospital_failure_trauma_cause'].fillna(fill)

In [24]:
# Creating a while loop that builds a list for all the cases where the trauma cause is not unknown and the 
# trauma count is zero
x=0
placholder_lst =[]
while x < lenght1 :
    if df.loc[x, 'mortality_hospital_failure_trauma_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_trauma_count'] < 1:
        placholder_lst.append(1)
       
    elif df.loc[x, 'mortality_hospital_failure_trauma_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_trauma_count'] > 0:
            placholder_lst.append(0)
    else:
        placholder_lst.append(0)
    x += 1


In [25]:
# Creating a while loop that builds a list for all the cases where the cardiovascular cause is not unknown and the 
# cardiovascular count is zero
x=0
placholder_lst2 =[]
while x < lenght1 :
    if df.loc[x, 'mortality_hospital_failure_cardiovascular_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_cardiovascular_count'] < 1:
        placholder_lst2.append(1)
       
    elif df.loc[x, 'mortality_hospital_failure_cardiovascular_cause'] != 'unknown' and df.loc[x, 'mortality_hospital_failure_cardiovascular_count'] > 0:
            placholder_lst2.append(0)
    else:
        placholder_lst2.append(0)
    x += 1


In [26]:
# Imputing 'power_outage_mortality '
# Very inconsistent data. Only 10 observations with power outage mortality yes and some of those
# with power outage mortality yes have no power outage death counts
# The remaining ones with power outage mortality no will be imputed as 0
def fill_na_power_mortality (data): 
    data.loc[(data['power_outage_deaths_count'].isnull()) & (data['power_outage_mortatility']=='No'), 'power_outage_deaths_count'] = 0
    return data
fill_na_power_mortality(df)
fill=0
df['power_outage_deaths_count'] = df['power_outage_deaths_count'].fillna(fill)

In [27]:
# Creating a while loop that builds a list for all the cases where the power outage mortality
# is not no and the mortality count is zero
x=0
placholder_lst3 =[]
while x < lenght1 :
    if df.loc[x, 'power_outage_mortatility'] == 'Sí' and df.loc[x, 'power_outage_deaths_count'] < 1:
        placholder_lst3.append(1)
       
    elif df.loc[x, 'mortality_hospital_failure_cardiovascular_cause'] == 'Sí' and df.loc[x, 'power_outage_deaths_count'] > 0:
            placholder_lst3.append(0)
    else:
        placholder_lst3.append(0)
    x += 1


In [28]:
# Imputing 'power_outage_equipment_failure_specify '
# Many values are filled as 0, already where no equipment failure occured or it is unknown.
# We impute 0 for missing values here as well. Very different way of handling this column
# For data accuracy it might be good to turn this field into a dropdown for the survey
fill = '0'
df['power_outage_equipment_failure_specify'] = df['power_outage_equipment_failure_specify'].fillna(fill)
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'ninguno'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'Ninguno'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'NINGUNO'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'No'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'no hubo cortes de energia electrica'] = "0"
df['power_outage_equipment_failure_specify'][df.power_outage_equipment_failure_specify == 'No hubo falla'] = "0"


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Applications/anaconda3/lib/pyt

In [29]:
# Creating a while loop that builds a list for all the cases where the power generator avalability
# is fully funtional but equipment failure occured because of power outages

x=0
placholder_lst4 =[]
while x < lenght1 :
    if df.loc[x, 'power_generator_available'] == 'Funciona todos los días' and df.loc[x, 'power_outage_equipment_failure_specify'] != '0':
        placholder_lst4.append(1)
       
    elif df.loc[x, 'power_generator_available'] == 'Funciona todos los días' and df.loc[x, 'power_outage_equipment_failure_specify'] != '00':
        placholder_lst4.append(1)
    else:
        placholder_lst4.append(0)
    x += 1


<h4>Creating logical inconsistency analysis for data quality</h4>

This section combines the previously created inconsistency flag lists into a logical inconsistency group for the data quality index. At the moment it only looks at possible inconsistencies in the mortality rate and power outage failure. With time it can be expanded to look for further data inconsistencies, especially in full text columns


In [30]:
#Creating dataframes from the previously created inconsistency lists
df6 = pd.DataFrame(placholder_lst)
df7 = pd.DataFrame(placholder_lst2)
df8 = pd.DataFrame(placholder_lst3)
df13 = pd.DataFrame(placholder_lst4)

#Combining dataframes into a index column with values between 0 and 1
df9 = (df6+df7+df8+df13)/4

In [31]:
#Adding it to the dataframe
df["logical_inconsistency"] = df9

<h4>Creating a strike free column</h4>

Creating a column that combines the missing value columns for all affected by strike columns to create a column that shows every week that is affected by any protests or strikes

In [32]:
# Creating #Strike_free colummn
# Creating a coulumn to see if there where any strikes by combining the missing value columns of the 
# different strike columns. If all values are missing there was no strike

df1 = []
df1 = (df['m_strike_other_affected'] + df['m_strike_patients_affected'] + df['m_strike_other_staff_affected'] 
+ df['m_strike_nurses_affected']+ df['m_strike_medical_staff_affected'])



no =[5]
yes=[0,1,2,3,4]
placeholder_lst = []

for strike in df1:
        if  strike in no:
            placeholder_lst.append(1)
            
        elif strike in yes:
            placeholder_lst.append(0)
        else:
            placeholder_lst.append(0)
        
df2 = pd.DataFrame(placeholder_lst)


# Concatenating with original DataFrame
df["strike_free"] = df2


<h4>Creating a duplicate indicator column for data quality</h4>

By creating a subset for the duplicates that will remain in the dataset, based on its unique timestamp we can create a list for all the duplicates that will remain in the dataset and turn it into a duplicate indicator column.  It will be used as part of the quality indicator column.

In [33]:
#Creating a subset of the duplicates that will stay in the dataset with keep first, since we will 
# keep the latest dataset. By removing the first data we keep the most recent data in the dataset

df10 = df.loc[df.duplicated(['report_date', 'hospital_code'],keep='first'), ['timestamp','report_date', 'hospital_code']]
duplicate_list = df10['timestamp'].tolist()
duplicate_list
placeholder_lst3=[]

for x in df['timestamp']:
        if  x in duplicate_list:
            placeholder_lst3.append(1)
        else:
            placeholder_lst3.append(0)
              
df11 = pd.DataFrame(placeholder_lst3)


# Concatenating with original DataFrame
df['duplicates']=df11

<h4> Removing duplicates from dataset</h4>

Removing duplicates from dataset using th predefined later duplicated hospital function.

In [34]:
# Removing duplicates
later_duplicated_hospital(df)

,Unnamed: 0,timestamp,report_week,operability_icu,power_outage_equipment_failure_specify,power_generator_available,power_outage_mortatility,power_outage_deaths_count,mortality_hospital_failure_cardiovascular_count,mortality_hospital_failure_cardiovascular_cause,mortality_hospital_failure_trauma_count,mortality_hospital_failure_trauma_cause,strike_medical_staff_affected,strike_nurses_affected,operability_icu_p,strike_other_staff_affected,strike_patients_affected,strike_other_affected,staff_violence_affected_reasons,epidemiological_emergency_suspected_diseases,rrt_avail_high_flow_catheters,rrt_avail_blood_tests_hiv_hvb_hvc_vdr,rrt_avail_immediate_access_urea_reduction_bun,nCoV_face_mask_avail,nCoV_respiratory_isolation_protocol_avail,operability_er,nCoV_isolation_area_avail,operability_sx,operability_lab,operability_uls,operability_ct_mri,operability_xr,er_avail_adrenalin,er_avail_atropine,hospital_code,er_avail_dopamine,er_avail_cephalosporins_betalactams,er_avail_aminoglycosides_quinolone,er_avail_vancomycin_clindamycin,er_avail_lidocaine,er_avail_minor_opioids,er_avail_major_opioids,er_avail_iv_fluids,er_avail_diazepam_dph,er_avail_heparin,federal_entity,er_avail_steroids,er_avail_insulin,er_avail_asthma,er_avail_blood_pressure,er_avail_defibrillator,er_avail_ott_intubation,er_avail_catheter,er_avail_oxygen_suction,sx_avail_minor_opioids,sx_avail_major_opioids,hospital_type,sx_avail_anesthetic_gases,sx_avail_anesthetics_iv,sx_avail_relaxants,sx_avail_ott_intubation,sx_avail_patient_lingerie_kit,sx_avail_disposables_mask_gloves_gown,sx_avail_oxygen_suction,rrt_avail,rrt_operability,rrt_date_stopped_operability,administrative_entity,rrt_reason_stopped_operability,rrt_avg_daily_patients,rrt_peritoneal_count,rrt_peritoneal_reason_not_performed,rrt_num_hemodialysis,rrt_num_hemodialysis_acute,rrt_num_hemodialysis_chronic,rrt_num_hemodialysis_equipments,rrt_num_hemodialysis_equipments_operability,rrt_hemodialysis_avail_filter,arch_beds_count,rrt_hemodialysis_avail_lines,rrt_hemodialysis_avail_kit_hemodialysis,rrt_hemodialysis_avail_iron,rrt_hemodialysis_avail_b_complex,rrt_hemodialysis_avail_calcium,rrt_hemodialysis_avail_zemblar,rrt_reverse_osmosis_unit_operability,rrt_staff_nephrology,rrt_staff_md,rrt_staff_resident,op_beds_count,rrt_staff_nurse,rrt_staff_nurse_nephrologist,nutr_avail,nutr_operability,nutr_date_stopped_operability,nutr_reason_stopped_operability,nutr_num,nutr_daily_freq_meal,nutr_quality,nutr_freq_milk_formulas,op_beds_er_count,pneumonia_antibiotic_therapy_latency,mi_thrombolytic_treatment_latency,er_staff_residents_and_rural_day_on_call,er_staff_specialist_day_on_call,er_staff_mic_day_on_call,er_staff_nurse_day_on_call,er_staff_non_professional_nurse_day_on_call,er_staff_residents_and_rural_night_on_call,er_staff_specialist_night_on_call,er_staff_mic_night_on_call,op_pavilions_count,er_staff_nurse_night_on_call,er_staff_non_professional_nurse_night_on_call,wash_failure_icu,wash_failure_er,wahs_failure_sx,power_outage,power_outage_avg_failures_per_day,power_outage_days_count,power_outage_avg_duration,power_outage_equipment_failure,m_power_outage_equipment_failure_specify,m_power_outage_deaths_count,m_mortality_hospital_failure_cardiovascular_count,m_mortality_hospital_failure_cardiovascular_cause,m_mortality_hospital_failure_trauma_count,m_mortality_hospital_failure_trauma_cause,m_strike_medical_staff_affected,m_strike_nurses_affected,m_strike_other_staff_affected,m_strike_patients_affected,m_strike_other_affected,m_staff_violence_affected_reasons,m_epidemiological_emergency_suspected_diseases,m_rrt_avail_high_flow_catheters,m_rrt_avail_blood_tests_hiv_hvb_hvc_vdr,m_rrt_avail_immediate_access_urea_reduction_bun,m_nCoV_face_mask_avail,m_nCoV_respiratory_isolation_protocol_avail,m_nCoV_isolation_area_avail,m_rrt_operability,m_rrt_date_stopped_operability,m_rrt_reason_stopped_operability,m_rrt_avg_daily_patients,m_rrt_peritoneal_count,m_rrt_peritoneal_reason_not_performed,m_rrt_num_hemodialysis,m_rrt_num_hemodialysis

In [35]:
#Extra information on the dataset after duplication
# Comparing number of observations before and after duplication
removed_rows = lenght1 - len(df)
# Removed duplicates as a percentage of observations
duplicateperc = removed_rows/ lenght1

<h4>Imputing missing values</h4><br>
Analysing missing values and imputing appropriate values for each column as described in the functions

<h5>Strikes</h5><br>
Strikes effect different groups differently. It might be that some groups are simply not affected 
by the strikes or that the effect is different on the different groups. We might need to create a column to see if there where any strikes in a given week by defining a column that combines the values for all strike columns into one to see if there are any protests or not.

In [36]:
# Imputing 'strike_medical_staff_affected'
# This variable only has two answers Protests with temporary suspension or Protests without suspension
# We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_medical_staff_affected'] = df['strike_medical_staff_affected'].fillna(fill)

In [37]:
# Imputing 'strike_nurses_affected'
# This variable only has two answers Protests with temporary suspension or Protests without suspension
# We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_nurses_affected'] = df['strike_nurses_affected'].fillna(fill)

In [38]:
# Imputing 'strike_other_staff_affected'
# This variable only has three answers Protests with temporary suspension, Protests without suspension
# or Protests with complete stop of activities. We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_other_staff_affected'] = df['strike_other_staff_affected'].fillna(fill)

In [39]:
# Imputing 'strike_patients_affected'
# This variable only has three answers Protests with temporary suspension, Protests without suspension
# or Protests with complete stop of activities. We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_patients_affected'] = df['strike_patients_affected'].fillna(fill)

In [40]:
# Imputing 'strike_other_affected'
# This variable only has two answers Protests with temporary suspension or Protests without suspension
# We can therfore assume that the null values are those without protests
fill = 'Sin protestas'
df['strike_other_affected'] = df['strike_other_affected'].fillna(fill)

<h5>Coronavirus</h5>

The impending epidemic of the Coronavirus is new; thus, we are assuming that the columns related to the Coronavirus were not applicable before the first entry on January 23rd, 2020. Null values in these columns before January 23rd were therefore filled as "Not applicable".

In [41]:
#Coronavirus columns were not applicable before January 23

for row in df["timestamp"]:
    if row <='2020-01-23T19:11:54.191Z':
        df["nCoV_face_mask_avail"] = df["nCoV_face_mask_avail"].fillna("Not Applicable")
        df["nCoV_isolation_area_avail"] = df["nCoV_isolation_area_avail"].fillna("Not Applicable")
        df["nCoV_respiratory_isolation_protocol_avail"] = df["nCoV_respiratory_isolation_protocol_avail"].fillna("Not Applicable")
        

<h5>Nutrition Services</h5>

If there is no nutrition service in a hospital at the time of data entry, then the subsequent nutrition service-related columns would not be applicable to this entry. Thus, if the personnel answers "No" to the question "Is there a nutrition service in your hospital?", then the rest of the "nutr" cells will be filled with "Not applicable". 



In [42]:
# Imputing nutrition service availability
# inputing not applicable in related columns for nutrition service not available

df.loc[949,'nutr_avail'] = 'Sí' # Correcting incorrect data input in row 949 (The only cell where the answer was "No", but subsequent nutrition services-related cells were answered)
df['nutr_operability'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_date_stopped_operability'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_reason_stopped_operability'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_num'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_daily_freq_meal'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_quality'][df.nutr_avail == 'No'] = "Not applicable"
df['nutr_freq_milk_formulas'][df.nutr_avail == 'No'] = "Not applicable"


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Applications/anaconda3/lib/python3.7/site-packages/ip

<strong>Applicability of Nutrition service<strong>
    
If a nutrition service in a hospital is operational, then information for the date and reason that the service stopped operability is not applicable. Thus, if the answer for "Is the nutrition service operational?" is "Yes", then the "nutr_date_stopped_operability" and "nutr_reason_stopped_operability" will be filled with "Not applicable".

In [43]:
# If the nutrition service is operational, then there would be no inputs for 
# "Date since the nutrition service was not operational" and "Reason the nutrion service is not operational"

df.loc[994,'nutr_operability'] = 'No'
df.loc[951,'nutr_operability'] = 'No'

df['nutr_date_stopped_operability'][df.nutr_operability == 'Sí'] = "Not applicable"
df['nutr_reason_stopped_operability'][df.nutr_operability == 'Sí'] = "Not applicable"


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
#For the rest of the Nutrition Service cells, values cannot be accurately imputed, so we fill them with "Unknown".
# Filling in the rest of the nutrition service data with "Missing" because 
# the values cannot be accurately imputed

df["nutr_num"] = df["nutr_num"].fillna("Missing")
df["nutr_daily_freq_meal"] = df["nutr_daily_freq_meal"].fillna("Missing")
df["nutr_quality"] = df["nutr_quality"].fillna("Missing")
df["nutr_freq_milk_formulas"] = df["nutr_freq_milk_formulas"].fillna("Missing")

<h5>Dialysis Services</h5>

If there is no dialysis service in a hospital at the time of data entry, then the subsequent dialysis-related columns would not be applicable to this entry. Thus, if the personnel answers "No" to the question "Is there dialysis service in your hospital?", then the rest of the related "rrt" cells will be filled with "Not applicable". 

In [45]:
# If there is no dialysis service in a hospital, then the rest of the 
# dialysis-related rows would not be applicable
df['rrt_avail_immediate_access_urea_reduction_bun'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_operability'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_date_stopped_operability'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_reason_stopped_operability'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_avg_daily_patients'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_peritoneal_count'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_peritoneal_reason_not_performed'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis_acute'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis_chronic'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_num_hemodialysis_equipments'][df.rrt_avail == 'No'] = "Not applicable"
df['rrt_reverse_osmosis_unit_operability'][df.rrt_avail == 'No'] = "Not applicable"


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

If a dialysis service in a hospital is operational, then information for the date and reason that the service stopped operability is not applicable. Thus, if the answer for "Is the dialysis service operational?" is "Yes", then the "rrt_date_stopped_operability" and "rrt_reason_stopped_operability" will be filled with "Not applicable".

In [46]:
# If the dialysis service is operational, then there would be no inputs for 
# "Date since the dialysis service was not operational" and 
# "Reason the dialysis service is not operational"
df['rrt_date_stopped_operability'][df.rrt_operability  == 'Sí'] = "Not applicable"
df['rrt_reason_stopped_operability'][df.rrt_operability  == 'Sí'] = "Not applicable"

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [47]:
#For the rest of the Dialysis Service cells, values cannot be accurately imputed, so we fill them with "Unknown".
# Filling the rest of the dialysis missing cells with "Unknown" because
# they cannot be accurately imputed
df['rrt_operability'] = df['rrt_operability'].fillna("Unknown")
df['rrt_date_stopped_operability'] = df['rrt_date_stopped_operability'].fillna("Unknown")
df['rrt_reason_stopped_operability'] = df['rrt_reason_stopped_operability'].fillna("Unknown")
df['rrt_avg_daily_patients'] = df['rrt_avg_daily_patients'].fillna("Unknown")
df['rrt_peritoneal_count'] = df['rrt_peritoneal_count'].fillna("Unknown")
df['rrt_peritoneal_reason_not_performed'] = df['rrt_peritoneal_reason_not_performed'].fillna("Unknown")
df['rrt_num_hemodialysis'] = df['rrt_num_hemodialysis'].fillna("Unknown")
df['rrt_num_hemodialysis_acute'] = df['rrt_num_hemodialysis_acute'].fillna("Unknown")
df['rrt_num_hemodialysis_chronic'] = df['rrt_num_hemodialysis_chronic'].fillna("Unknown")
df['rrt_num_hemodialysis_equipments'] = df['rrt_num_hemodialysis_equipments'].fillna("Unknown")
df['rrt_num_hemodialysis_equipments_operability'] = df['rrt_num_hemodialysis_equipments_operability'].fillna("Unknown")
df['rrt_reverse_osmosis_unit_operability'] = df['rrt_reverse_osmosis_unit_operability'].fillna("Unknown")



In [48]:
# Filling in the rest of the nutrition service data with "Missing" because 
# the values cannot be accurately imputed

df["nutr_num"] = df["nutr_num"].fillna("Missing")
df["nutr_daily_freq_meal"] = df["nutr_daily_freq_meal"].fillna("Missing")
df["nutr_quality"] = df["nutr_quality"].fillna("Missing")
df["nutr_freq_milk_formulas"] = df["nutr_freq_milk_formulas"].fillna("Missing")


In [49]:
# Imputing 'staff_violence_affected_reasons'
# This is a multiselection row without a count. We can assume that the blank fields are from weeks
# without incident  
fill = 'None reported'
df['staff_violence_affected_reasons'] = df['staff_violence_affected_reasons'].fillna(fill)

In [50]:
# Imputing 'epidemiological_emergency_suspected_diseases'
# This is a multiselection row without a count. We can assume that the blank fields are from weeks
# without incident  
fill = 'None reported'
df['epidemiological_emergency_suspected_diseases'] = df['epidemiological_emergency_suspected_diseases'].fillna(fill)

<h4>Creating missing values percentage for data quality</h4>

Creating a missing value percentage for data quality by summing up the missing value per column and dividing the by the number of columns that can have missing values. It creates a percentage of missing values of potential missing values that is a good indicator for data quality.

In [51]:
# Creating Number of missing values and percentage of potential missing values column as part of a
# data quality index per row.

df4 = []
df4 = (df['m_strike_other_affected'] + df['m_strike_patients_affected'] + df['m_strike_other_staff_affected'] 
+ df['m_strike_nurses_affected']+ df['m_strike_medical_staff_affected']+ df['m_nutr_freq_milk_formulas']
+ df['m_nutr_quality']+ df['m_nutr_daily_freq_meal']+ df['m_nutr_num']
+ df['m_nutr_reason_stopped_operability'] + df['m_nutr_date_stopped_operability']+ df['m_nutr_operability']
+ df['m_rrt_staff_nurse_nephrologist']+ df['m_rrt_staff_nurse']+ df['m_rrt_staff_resident'] 
+ df['m_rrt_staff_md']+ df['m_rrt_staff_nephrology']+ df['m_rrt_reverse_osmosis_unit_operability']
+ df['m_rrt_hemodialysis_avail_zemblar']+ df['m_rrt_hemodialysis_avail_calcium']+ df['m_rrt_hemodialysis_avail_b_complex']
+ df['m_rrt_hemodialysis_avail_iron']+ df['m_rrt_hemodialysis_avail_kit_hemodialysis']+ df['m_rrt_hemodialysis_avail_lines'] 
+ df['m_rrt_hemodialysis_avail_filter']+ df['m_rrt_num_hemodialysis_equipments_operability']+ df['m_rrt_num_hemodialysis_equipments']            
+ df['m_rrt_num_hemodialysis_chronic']+ df['m_rrt_num_hemodialysis_acute']+ df['m_rrt_num_hemodialysis']
+ df['m_rrt_peritoneal_reason_not_performed']+ df['m_rrt_peritoneal_count']+ df['m_rrt_avg_daily_patients']      
+ df['m_rrt_reason_stopped_operability']+ df['m_rrt_date_stopped_operability']+ df['m_rrt_operability']           
+ df['m_nCoV_isolation_area_avail']+ df['m_nCoV_respiratory_isolation_protocol_avail']+ df['m_nCoV_face_mask_avail']         
+ df['m_rrt_avail_immediate_access_urea_reduction_bun']+ df['m_rrt_avail_blood_tests_hiv_hvb_hvc_vdr']+ df['m_rrt_avail_high_flow_catheters']        
+ df['m_epidemiological_emergency_suspected_diseases']+ df['m_staff_violence_affected_reasons']+ df['m_mortality_hospital_failure_trauma_cause']   
+ df['m_mortality_hospital_failure_trauma_count']+ df['m_mortality_hospital_failure_cardiovascular_cause']+ df['m_mortality_hospital_failure_cardiovascular_count'] 
+ df['m_power_outage_deaths_count']+ df['m_power_outage_equipment_failure_specify'])
df5 = df4 /50

df["number_of_missing_values"] = df4
df["percentage_of_potential_missing_values"] = df5

<h4>Creating data quality index</h4>

The data quality index is created from the duplicate column, logical inconsistency and percentage of potential missing values.
It is calculated as 1 minus the positive values for missing data, duplicates and inconsistencies.
The weighing for the different columns at the moment is 3 for missing values, 2 for duplicates and 1 for logical inconsistencies. Missing values is weighted as much as duplicates and inconsistencies combined since it is the most common occurrence of data quality issues. Inconsistencies are weighted the least, since this column has yet to be expanded and since this data is more related to mistakes in filling out the form than bad data in general. These weights can be changed depending on your analysis by changing the code below.


In [52]:
df['data_quality_index'] =(1-((2*df['duplicates']+df['logical_inconsistency']+3*df["percentage_of_potential_missing_values"])/6))

<h4>Ordinal encoding columns for Tableau</h4>

Values for supply, equipment, operability, public services and strikes are encoded as ordinal scales between 1 and 0 in order to show percentages in Tableau.

In [53]:
#Ordinal_encoding_supply
one_hot_supply = ['rrt_staff_nurse_nephrologist','rrt_staff_nurse', 'rrt_hemodialysis_avail_lines', 'rrt_hemodialysis_avail_kit_hemodialysis', 'rrt_hemodialysis_avail_iron', 'rrt_hemodialysis_avail_b_complex','rrt_hemodialysis_avail_calcium','rrt_hemodialysis_avail_zemblar','rrt_staff_nephrology','rrt_staff_md','rrt_staff_resident','er_avail_adrenalin','er_avail_atropine', 'er_avail_dopamine','er_avail_cephalosporins_betalactams','er_avail_aminoglycosides_quinolone', 'er_avail_vancomycin_clindamycin','er_avail_lidocaine','er_avail_minor_opioids','er_avail_major_opioids', 'er_avail_iv_fluids','er_avail_diazepam_dph','er_avail_heparin','er_avail_steroids','er_avail_insulin', 'er_avail_asthma','er_avail_blood_pressure','er_avail_defibrillator','er_avail_ott_intubation', 'er_avail_catheter','er_avail_oxygen_suction','sx_avail_minor_opioids','sx_avail_major_opioids','sx_avail_anesthetic_gases','sx_avail_anesthetics_iv','sx_avail_relaxants','sx_avail_ott_intubation', 'sx_avail_patient_lingerie_kit','sx_avail_disposables_mask_gloves_gown','sx_avail_oxygen_suction', 'rrt_avail_high_flow_catheters', 'rrt_avail_blood_tests_hiv_hvb_hvc_vdr', 'rrt_avail_immediate_access_urea_reduction_bun']
df [one_hot_supply] = df[one_hot_supply].replace({"Nunca ha existido": np.nan, 
    'No hubo': 0,
    'Entre 1 y 2 días': 0.33, 
    'Entre 3 y 5 días':0.66,
    'Todos los días':1})

#Ordinal_encoding_operability
one_hot_operability = ['operability_icu', 'operability_icu_p','operability_er','operability_sx','operability_lab', 'operability_uls','operability_ct_mri','operability_xr']
df [one_hot_operability] = df[one_hot_operability].replace({"Nunca ha existido": np.nan, 
    'No existe': 0,
    'No operativa': 0,
    'Menos de 3 de días':0.33, 
    'Entre 3 y 5 días':0.66,
    'Todos los días':1})

#Ordinal_encoding_operating_machines
one_hot_machines = ['power_generator_available']
df [one_hot_machines] = df[one_hot_machines].replace({"Nunca ha habido": np.nan, 
    'Hay pero no funciona': 0,
    'Funciona menos de 3 días': 0.33,
    'Funciona entre 3 y 5 días': 0.66, 
    'Funciona todos los días':1})

#Ordinal_encoding_encoding_washing_conditions
one_hot_wash = ['wash_failure_icu', 'wash_failure_er', 'wahs_failure_sx']
df [one_hot_wash] = df[one_hot_wash].replace({'No hubo agua ningún dia': 0, "NNo hubo agua ningún dia": 0, 
    '< 3 días, sin soporte alterno (cisternas)': 0.16,
    '< 3 días, con soporte alterno': 0.33,                                          
    '3 a 5 días , sin soporte  alterno': 0.50,
    '3 a 5 días, con soporte alterno':0.66,
    'Funciona entre 3 y 5 días': 0.83,
    'Hubo agua todos los días': 1})

#Ordinal_encoding_encoding_power_outage
power_out = ['power_outage_days_count']
df [power_out] = df[power_out].replace({"Menos de 3 días": 0.33, 
    'Entre 3 y 5 días': 0.66,
    'Todos los días': 1})    

#Ordinal_encoding_strikes
one_hot_strike = ['strike_medical_staff_affected', 'strike_nurses_affected','strike_other_affected']
df [one_hot_strike] = df[one_hot_strike].replace({ 
    'Sin protestas': 0,
    'Protestas  sin suspensión de actividades': 0.5,
    'Protestas con suspensión temporal  de actividades': 1})

one_hot_strike = ['strike_other_staff_affected','strike_patients_affected']
df [one_hot_strike] = df[one_hot_strike].replace({ 
    'Sin protestas': 0,
    'Protestas  sin suspensión de actividades': 0.33,
    'Protestas con suspensión temporal  de actividades': 0.66,
    'Huelga (suspendidas todas  las actividades)':1})


<h4>Export to csv for import to Tableau</h4>

In [ ]:
df.to_csv('public_services.csv')

<h3>Next steps</h3><br>
The data cleaning and data quality analysis can be further expanded and the Tableau functions can be expanded and the layout can be changed according to your wishes. 